# Getting the data from SQL

How we got to the data we are demonstrating on.

In [2]:
# establish connection to the server where the data lives
import pyodbc

server = '<hidden>'
database= '<hidden>'
user='<hidden>'
table='grocery_sales'
passwd='<hidden>' 

conndrv = 'DRIVER={SQL Server Native Client 11.0};'
connpar = 'SERVER={{{0}}};DATABASE={{{1}}};UID={{{2}}};PWD={{{3}}}'
connstr = conndrv + connpar.format(server, database, user, passwd)

conn = pyodbc.connect(connstr)
crsr = conn.cursor()

In [ ]:
# DEAD END
query = "select g.* from grocery_sales"

# crsr.execute(query)
# rows = crsr.fetchall()

Why a dead end? A dozen cells below, this data proves to be problematic because it contains many short series for products that were sold only once or twice, or were seasonal specials. Instead, we have returned from below and had to re-pull only the data that is machine-learnable. This gives us a problem: we will not be able to make forecasts for those items we left behind! We will fix that problem later, but let's make a note.

*Practical problem 1*: we left some grains behind. How are we going to forecast them? We'll return to that.

# Technique 0: Get only machine-learnable data from source

In [4]:
# We will load a smaller subset for the tutorial that will run faster.
# But this is what actually happens, so the code is here for you.

# Pull things that have been sold for at least 100 days (not consecutive).
# This is a good way to filter the data because it matches the requirement
# of cross-validation: N periods avaiable (It's OK if the value is 0.)

# Do the filtering in a small DB rather than in a big pandas job
# to avoid out-of-memory experiences.
query = "select g.* \
        from grocery_sales g join \
            ( \
            select Item, Site, Channel, count(1) as count \
            from grocery_sales \
            group by Item, Site, Channel \
            having DATEDIFF(day, min(SalesDate), max(SalesDate)) > 100 \
            ) long_grains \
            on (long_grains.Item = g.Item \
            and long_grains.Site = g.Site \
            and long_grains.Channel = g.Channel ) \
            and g.Item like 'A%'"    # only for the tutorial, to get a small set

crsr.execute(query)
rows = crsr.fetchall()

# parse the data coming from a database
import pandas as pd
SQLdata = pd.DataFrame(columns=["SalesDate", "Item", "Site", "Channel", "Quantity"]) 
SQLdata["SalesDate"] = [i[0] for i in rows]
SQLdata["Item"] = [i[1] for i in rows]
SQLdata["Site"] = [i[2] for i in rows]
SQLdata["Channel"] = [i[3] for i in rows]
SQLdata["Quantity"] = [i[4] for i in rows]

SQLdata.head()

,SalesDate,Item,Site,Channel,Quantity
0,2015-11-22,A00968,1B3B,811B,9.0
1,2015-11-27,A00968,1B3B,811B,6.0
2,2015-12-24,A00968,1B3B,811B,6.0
3,2016-09-16,A00968,1B3B,811B,6.0
4,2016-09-25,A00968,1B3B,811B,1.0


In [15]:
SQLdata.describe(include='all')

,SalesDate,Item,Site,Channel,Quantity
count,1262885,1262885,1262885,1262885,1.262885e+06
unique,2057,348,11,9,NaN
top,2017-05-26,A53634,82DC,811B,NaN
freq,996,43416,359820,391703,NaN
mean,NaN,NaN,NaN,NaN,1.201671e+01
std,NaN,NaN,NaN,NaN,2.587775e+01
min,NaN,NaN,NaN,NaN,1.000000e+00
25%,NaN,NaN,NaN,NaN,5.000000e+00
50%,NaN,NaN,NaN,NaN,8.000000e+00
75%,NaN,NaN,NaN,NaN,1.200000e+01


In [11]:
grain_colnames = ['Item', 'Site', 'Channel']
time_colname = 'SalesDate'
target_colname = 'Quantity'

In [12]:
# How many series do we have?
guppy = SQLdata.groupby(grain_colnames)

print("Rows pulled : " + str(len(SQLdata)))
print("Distinct time series : " + str(len(guppy)))

Rows pulled : 1262885
Distinct time series : 6139


In [10]:
SQLdata.to_csv('tutorial_data.csv.bz2', compression='bz2', index=False)